# Data Mining: Extracting Dates from Messy Medical Records

Extracts dates from messy medical notes using regular expressions on Pandas

Each data instance corresponds to a medical note containing a date written in one of many formats.

Examples of date encoding variants in the data are:
* 03/20/2009; 03/20/09; 3-20-09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* 3/09, 2009

Including some typos and other confounding errors/formats. 

In [99]:
import pandas as pd
doc = []
with open('medical_notes.txt') as file:
    for line in file:
        doc.append(line)
df = pd.Series(doc)

pd.set_option('display.max_colwidth', 90)
pd.set_option('display.max_rows', 100)

### Sample of original data

In [87]:
import random
sample = random.sample(range(1, 500), 10)
df[sample]

171                                              04 Oct 1972 Total time of visit (in minutes):\n
72     Lithium 0.25 (7/11/77).  LFTS wnl.  Urine tox neg.  Serum tox + fluoxetine 500; otherw...
378    The pt. is a 45 Y/O F who comes in to discuss her sexual problems-low libido.  Factors...
24                                             07/25/1984 CPT Code: 90791: No medical services\n
455                                        sHemmorage caused by probe in 1984 Medical History:\n
269            passive SI only (most recent episode end of July 1992) If Yes, please describe:\n
2                                       sshe plans to move as of 7/8/71 In-Home Services: None\n
159    .The patient brought himself to APS on 26 May 2001 for ONE MONTH HISTORY of recurrence...
177    22 year old single Caucasian/Latino woman, unemployed Cook recent college graduate, li...
175     .On 21 Oct 1983 patient was discharged from Scroder Hospital after EIGHT DAY ADMISSION\n
dtype: object

### Define regex patterns for date search in numerical format 

It prioritizes more correct and complete formats, if no date is found, it then will search alternative, less strict and incrementaly more inclusive formats

In [88]:
pattern0 = r'\d{1,2}[-/]\d{1,2}[/-]\d{4}'
pattern1 = r'\d{1,2}[-/]\d{1,2}[/-]\d{2}'
pattern2 = r'\d{1,2}?[-/]?\d{1,2}[/-]\d{4}'
pattern3 = r'\d{0,2}[-/]?\d{1,2}[/-]\d{2,4}'

num_patterns = pattern1, pattern2, pattern3

### Initialize series of dates, then fill blanks step by step using numeric patterns

In [89]:
dates = df.str.findall(pattern0)

for pat in num_patterns:
  blank_idx = dates[dates.str.len() == 0].index
  new_dates = df[blank_idx].str.findall(pat)
  add_idx = new_dates[new_dates.str.len() > 0].index
  dates[add_idx] = new_dates[add_idx]

### Create month abbreviations string for regex

In [90]:
import calendar
mo_nums = range(1,13) 
months = [calendar.month_abbr[x] for x in mo_nums]
mo_str = '|'.join(str(mo) for mo in months)
print(mo_str)

Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec


### Find dates in alphanumeric format in the original data and create a new series
Results from here will take precedent over numeric only formats

In [91]:
pattern_alnum = (r'\d{0,2}[.,]?\s*(?:' + mo_str + ')[a-z]*,?.?\s*\d{0,2}.?,?\s*\d{2,4}')
alnum_dates = df.str.findall(pattern_alnum)

### Add series to dataframe, fill values from alphanumeric results, inspect unmatched instances

In [92]:
idx_alnum = alnum_dates.index[alnum_dates.str.len() > 0]
df = pd.concat(dict(data = df, date = dates), axis=1)
df['date'][idx_alnum] = alnum_dates[idx_alnum]
idx_remain = df[df['date'].str.len() == 0].index
df['data'][idx_remain].head()

455                                        sHemmorage caused by probe in 1984 Medical History:\n
456    sHas been at MYH since his treaters in NE retired in 2000. Was seen in NE for 20 years...
457    Pt joined Army reserves in 2001 and has 3 years left in this commitment.-Mental Status...
458    one sister from whom he is estranged due to her opiate dependence, legal conflict over...
459           sSince 1998. Prior medication trials (including efficacy, reasons discontinued):\n
Name: data, dtype: object

Remaining unmatched records contain dates that inlclude year only

In [93]:
pattern4 = r'\d{4}'
df['date'][idx_remain] = df['data'][idx_remain].str.findall(pattern4)

### Initialize new date series to be cleaned and normalized
Define patterns to be replaced/modified  
Define replacements

In [94]:
df['fixed_date'] = [df['date'][i][0].strip().rstrip() for i in range(0,len(df))]

pp1 = '/'
pp2 = '(?<=-)(?=\d{2}$)'
pp3 = '(?=^\d{1}-)'
pp4 = '(?<=-)(?=\d{1}-)'
pp5 = '[^\w-]'
pp6 = '\s+'
pp7 = '^\W'
pp8 = '(?<=\d)(?=[A-Z])'
pp9 = '(?<=[A-Za-z]{3})[a-z]+'

post_process = pp1, pp2, pp3, pp4, pp5, pp6, pp7, pp8, pp9
repl = '-', '19', '0', '0', ' ', ' ', '', ' ', ''

### Make defined corrections/replacements, tranform to datetime format

In [95]:
for i, pat in enumerate(post_process):
  df['fixed_date'] = df['fixed_date'].str.replace(pat, repl[i])

df['date_time'] = pd.to_datetime(df['fixed_date'])

### Final dataframe contains original records and normalized dates in datetime format

In [96]:
final_df = df[['data', 'date_time']]
pd.set_option('display.max_colwidth', 90)
final_df.iloc[sample]

,data,date_time
171,04 Oct 1972 Total time of visit (in minutes):\n,1972-10-04
72,Lithium 0.25 (7/11/77). LFTS wnl. Urine tox neg. Serum tox + fluoxetine 500; otherw...,1977-07-11
378,The pt. is a 45 Y/O F who comes in to discuss her sexual problems-low libido. Factors...,1978-12-01
24,07/25/1984 CPT Code: 90791: No medical services\n,1984-07-25
455,sHemmorage caused by probe in 1984 Medical History:\n,1984-01-01
269,"passive SI only (most recent episode end of July 1992) If Yes, please describe:\n",1992-07-01
2,sshe plans to move as of 7/8/71 In-Home Services: None\n,1971-07-08
159,.The patient brought himself to APS on 26 May 2001 for ONE MONTH HISTORY of recurrence...,2001-05-26
177,"22 year old single Caucasian/Latino woman, unemployed Cook recent college graduate, li...",1990-01-18
175,.On 21 Oct 1983 patient was discharged from Scroder Hospital after EIGHT DAY ADMISSION\n,1983-10-21
